<a href="https://colab.research.google.com/github/cs-pub-ro/ML/blob/master/ML/homework/hw3/GM/Tema_3_Neural_Style_Transfer_Skel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tema 3 - A new Picasso is in town
## Neural Style Transfer
### Autori: 
* George Muraru

## 1. Scopul temei


Tema are ca scop folosirea rețelelor neurale convoluționale pentru a **modifica imagini prin preluarea stilului** de la o alta imagine.
Este o tehnica prin care se genereaza o noua imaginie folosind o imagine de baza (eng. *content image*) și o imagine a cărui stil (eng. *style image*) dorim să îl transferăm la imaginea de bază (rezultat numit în temă *generated image*). 


![result](https://raw.githubusercontent.com/cs-pub-ro/ML/master/homework/hw3/GM/img/colab/result.svg)



### 2. High Level Architecture

<img src="https://miro.medium.com/max/1294/1*ZgW520SZr1QkGoFd3xqYMw.jpeg" alt="High Level Architecture" width=1250/>

Reference: [Intuitive guide to neural style transfer](https://towardsdatascience.com/light-on-math-machine-learning-intuitive-guide-to-neural-style-transfer-ef88e46697ee)
 

### Descărcare bibloteci/fișiere de care vom avea nevoie

In [0]:
!pip install torch
!pip install torchvision
!pip install pillow

!wget -q -O starry_night.jpg https://github.com/cs-pub-ro/ML/blob/master/homework/hw3/GM/img/style/starry_night.jpg?raw=true
!wget -q -O cat.png https://github.com/cs-pub-ro/ML/blob/master/homework/hw3/GM/img/content/cat.png?raw=true

### Biblioteci de care vom avea nevoie

In [0]:
import torch
import torch.optim as optim
from PIL import Image
from torchvision import transforms, models
import matplotlib.pyplot as plt

### Hiper-parametrii



In [0]:
CUDA = False #@param {type: "boolean"}
ALPHA = 1 #@param {type: "slider", min: 0, max: 1e5, step: 1}
BETA = 1e3 #@param {type: "slider", min: 0, max: 1e5, step: 1}
IMG_SIZE = 512 #@param [128, 256, 512]

### Model

Pentru a putea extrage **stilul** și **conținutul** unei imagini ne vom folosi de o rețea neurală deja antrenată.

Noțiunea de a folosi o rețea pre-antrenată pentru a rezolva un alt task poartă denumirea de [Transfer Learning](https://ruder.io/transfer-learning/).

Într-o rețea convoluțională primele layere extrag feature-uri low-level (acestea le vom folosi să extragem **stilul**), iar layerele "mai apropiate de output" ne vor furniza **conținutul**.

Pentru acest lucru vom folosi o rețea VGGNet, mai specific VGG19.

![VGG19](https://www.researchgate.net/profile/Clifford_Yang/publication/325137356/figure/fig2/AS:670371271413777@1536840374533/llustration-of-the-network-architecture-of-VGG-19-model-conv-means-convolution-FC-means.jpg)

Parametrii rețelei vor trebui să ramână fixați la valorile inițiale pe parcursul rulării. Din acest motiv va trebui să nu facem update parametrilor în timpul optimizării.

Din rețeaua mai sus specificată vom folosi anumite layere (descrise mai jos) pentru a extrage feature-urile care ne interesează pentru cele 2 loss-uri.

In [0]:
# TODO 1.0: Load model and move it to CUDA if selected

# TODO 1.1: Freeze the parameters for the model

### Imagini folosite

Problema pe care dorim să o rezolvăm este de a genera **conținut** apropiat de o imagine de referință, conținut al cărui **stil** trebuie să fie apropiat de o altă imaginie.

Pentru acest lucru ne vom defini 3 imagini:
1. Imaginea a cărui conținut dorim să il copiem - notată $I_{C}$
2. Imaginea a cărui stil dorim să îl copiem - notată $I_{S}$
3. Imaginea generată (aceasta va avea inițial valori generate random) - notată $I_{G}$
  * această imagine va fi modificată în timpul optimizării

Intuitiv, soluția ar trebui să presupună rezolvarea a 2 probleme separate:
* Generare de **conținut**
* Generare de **stil**

Pentru a cunatifica cât de aproape suntem de o soluție (o imaginie generată perfect) vom folosi 2 loss-uri, specificate în continuare.

### Lucrul cu imagini

Un prim pas care trebuie realizat este deschiderea și afișarea unei imagini.
Ar trebui ca să avem 2 imagini descărcate: *starry_night.jpeg* și *cat.png*.

De asemnea, deoarece folosim o rețea neurală predefinită, va trebui să modificăm inputul astfel încât acesta să corespundă inputului așteptat de către rețea.



In [0]:
def get_image(img_path):
    """ Get an image ready to be run through the pretrained neural network

    Args:
        img_path (string): the path to the image

    Returns:
        A tensor with shape (B, H, W, C) where:
        * B - represents the batch size (SHOULD BE 1)
        * C - number of channels (3 - RGB)
        * H - the height of the image (it should be IMG_SIZE after transformation)
        * W - the width of the image (it should be IMG_SIZE after transformation)
    """
    img = Image.open(img_path).convert('RGB')

    '''
        TODO 1.0 - Use the transforms from torch library to do the following ops:
        * Resize to an image of (IMG_SIZE, IMG_SIZE)
        * Transform to a tensor (since the PIL library works with numpy arrays)
        * The tensor should have values in the range [0, 1]
        * Normalize using :
            * mean = [0.485, 0.456, 0.406]
            * std = [0.229, 0.224, 0.225]
            (because the pretrained model was trained using those values)
            (Formula: new_img = (img - mean) / std)
    '''

    # TODO 1.1: Add a size 1 dimension for the batch
    # Hint: unsqueeze

    # TODO 1.2: Move to cuda if support is enabled

    return img


def show_imgs(*imgs):
    """ Plot images received as parameters (on the same row with different
        collumn)

    Args:
        imgs (tensors representing the images): images to plot
    """
    plt.figure(figsize=(15,15))

    # TODO 2.0:
    # Define the inverse of the `normalize` and `toTensor` operations performed
    # in the `get_image` function

    for i, img in enumerate(imgs):

        # TODO 2.1: Detach, clone and move to CPU (the image)

        # TODO 2.2: Remove the `batch` dimension (the first one)
        # Hint: squeeze

        # TODO 2.3: Apply the inverse of the `normalize` operation

        plt.subplot(1, 2, i+1)

        # Remove the x, y coordinates from the plot and show the image

        plt.xticks([])
        plt.yticks([])
        plt.imshow(img)
    plt.show()

In [0]:
# Testing Area
img_cat = get_image("cat.png")
img_starry_night = get_image("starry_night.jpg")

assert isinstance(img_cat, torch.Tensor)
assert img_cat.shape == (1, 3, IMG_SIZE, IMG_SIZE)
assert not CUDA ^ img_cat.is_cuda

show_imgs(img_cat, img_starry_night)

### Content Loss

Loss-ul aceasta are rolul de a ne ajuta să recreăm **conținutul** unei imagini, el însemnând cât de mult diferă imaginea generată față de imaginea al cârui conținut dorim să îl "copiem".

După cum s-a menționat mai sus, activările (eng. activation map/feature map) straturilor convoluționale aflate spre finalul rețelei ne oferă informații mai high-level despre o imagine.

Putem folosi aceste activări pentru a calcula **content loss**:

$$
J_{content} = \frac{1}{N}\sum_{i,j}(a_{generated}^{i,j} - a_{content}^{i,j})^2
$$

Unde:
* $N$ - numărul de elemente din feature map-ul specific
* $a_{generate}$ - matricea activărilor specifice $I_{G}$
* $a_{content}$ - matricea activărilor specifice $I_{C}$

Pentru a obține feature map-ul se va face un **feedforward** cu imaginile:
* $I_{G}$ pentru a obține $a_{generated}$
* $I_{C}$ pentru a obține $a_{content}$

Stratul de convoluție a carui activări ne interesează este **conv4_2** (vedeți imaginea de mai sus de la VGG19).

In [0]:
def run_feedforward(img, model):
    ''' Do a feedforward step using the image given as parameter and keep track
    of the activations

    Args:
        img (Tensor): the image used for feedforward
        model (Model): the pretrained (and frozen) model

    Returns:
        A list containing the activations generated by passing the image through
    the model
    '''
    activations = []
    
    # TODO 3.0: Compute the activations (for each layer of the model) and
    # store them in the list

    return activations


def get_content_loss(features_content, features_generated):
    ''' Get the Content Loss

    Args:
        features_content ([Tensor]): the activations for I_C
        features_generatd ([Tensor]): the activations for I_G
    Returns:
        loss (double) - the computed loss for the content
    '''
    # TODO 4.0: Compute the content lost

    return loss

### Style Loss



#### **Gram Matrix (Style Matrix)**




Pentru a calcula *style loss-ul* va trebui să folosim *matricea Gram*, calculată pe activările anumitor straturi convoluționale.

Matricea Gram a unui set de vectori V = $(v_0, v_1, v_2,...,v_n)$ este dată de produsul matriceal dintre $V și V^t$.

Această matrice are rolul de a indica care set de feature-uri sunt corelate au valori mari împreună/sunt similare.

Mai multe detalii puteți găsi [aici](https://arxiv.org/pdf/1606.01286.pdf)


#### **Calcul Style Loss**

Style loss-ul va fi reprezentat de *distanța* între matricea gram specifică activărilor pentru imaginea $I_{G}$ și pentru imaginea $I_{S}$.

Pentru activările unui specific layer convoluțional, formula de calcul este următoarea:
$$
    J_{style}^{l} = \frac{1}{4N^2}\sum_{i,j}(G^{i,j}(a_{generated}) - G^{i,j}(a_{style}))^2
$$

Unde:
* l - layer-ul convoluțional pentru care se calculează loss-ul
* $N$ - numărul de elemente din feature map-ul specific
* $a_{generate}$ - matricea activărilor specifice $I_{G}$
* $a_{style}$ - matricea activărilor specifice $I_{style}$

Pentru a obține feature map-ul se va face un **feedforward** cu imaginile:
* $I_{G}$ pentru a obține $a_{generated}$
* $I_{S}$ pentru a obține $a_{style}$

Deoarece activările pentru $I_{G}$ au fost deja calculate de la pasul de *Contnt Loss* acestea **nu vor mai fi recalulate la acest pas**.

Dacă pentru calcularae *content loss-ului* am folosit un singur strat de convoluție, pentru a *captura* stilul unei imagini, vom folosi **mai multe straturi din rețeaua VGG19**, fiecare strat având o anumită pondere la *style loss-ul* final (vedeți cerința de mai jos).

In [0]:
def get_gram_matrix(X):
    ''' Compute the Gram Matrix for the tensor X (the activation map)
    
    Args:
        X (Tensor): Tensor with shape (B, F, H, W)
    
    Returns:
        Tensor representing the Gram Matrix
    '''
    g = None
    _, c, h, w = X.shape

    # TODO 5.0: `g` should be a matrix where:
    # * the rows -- nr of channels
    # * the columns -- elements for that specific channel
    # X_new = ...
    return g


def get_style_loss(features_style, features_generated):
    ''' Get the Style Loss

    Args:
        features_style ([Tensor]): the activations for I_S
        features_generatd ([Tensor]): the activations for I_G
    Returns:
        loss (double) - the computed loss for the content
    '''

    # TODO 5.1 Compute style loss

    return loss

### Generare imagine

Se vor folosi loss-urile definite anterior pentru a calcula un loss total și pentru a genera imaginea finală.

In [0]:
content_img = get_image("cat.png")
style_img = get_image("starry_night.jpg")

steps = 10000

generated_img = content_img.clone()

# Need to be set since this image will be updated using the optimizer
generated_img.requires_grad = True

# TODO 6.0: Get content and style features

optimizer = optim.LBFGS([generated_img.requires_grad_()])

it = [0]
for i in range(1, steps+1):
    def closure():
            optimizer.zero_grad()

            # TODO 6.1: Run feedforward for the generated_img

            # TODO 6.2: Compute content loss

            # TODO 6.3: Compute style loss

            loss = ALPHA * content_loss + BETA * content_loss
            loss.backward()

            it[0] += 1
            if it[0] % 500 == 0:
                print(f'Style Loss: {style_loss} Content Loss: {content_loss}')
                show_imgs(generated_img)

            return loss

    optimizer.step(closure)

### Cerințe


1. [0.5pct] Încărcați modelul pre-antrenat VGG19, mutați-l pe CUDA (daca există suport) și faceți ca antrenarea să nu modifice parametrii modelului.

2. [2pct] Scrieți codul necesar pentru a încarca o imagine și a o pregăti pentru a putea fi trimisă la rețeaua anterior încarcată.

    Mutați datele pe CUDA (daca există suport).

    Convertiți imaginea în formatul inițial pentru a putea fi afișată.


3. [1pct] Implementați funcția *run_feedforward* care trebuie să facă o trecere a datelor prin rețea.

4. [1pct] Calculați *Content Loss*.
   
   Folosiți layerul e convoluție: **conv4_2** pentru a extrage activarea necesară.

4. [2pct] Calculați *Style Loss*.
   
   Folosiți layerele următoare pentru a extrage *style loss-ul*: **conv1_1, conv2_1, conv3_1, conv4_1, conv5_1**.

    Fiecare strat va avea o pondere de 0.2.
$$
    J_{style} = \sum_{i}w_i  J_{style}^{i}
$$

5. [2.5pct] Completați codul necesar pentru a genera imaginea finală.
   
   Vedeți ce se întâmplă când modificați parametrul ALPHA, BETA, learning rate-ul, ponderile de la *Style Loss* pentru fiecare strat, straturile folosite pentru extragerea *stilului* și a *conținutului*.

    De asemenea, se poate experimenta și cu inițializarea $I_{G}$ cu valori random.

6. [1pct] Rulați programul pentru alte 3-4 perechi de imagini.

7. [Bonus - 2pct] Încercați să îmbunătățiți calitatea imaginii finale.

    Posibile metode: adaugare loss adițional, calculat loss-urile folosind o altă formulă, schimbare rețea, etc.
    Pentru aceasta trebuie să aveți în notebook o poză generată folosind metoda inițială și o altă poză cu noua metodă.

**Atenție**: Este în regulă dacă pozele generate conțin artefacte! De asemenea, anumiți parametrii funcționeaza mai bine pentru diferite poze.